Description: Shrimp , (Mexico), west coast, frozen, white, No. 1, shell-on, headless, 26 to 30 count per pound, wholesale price at New York.

Unit: US Dollars per Kilogram.

[Data](https://www.kaggle.com/mruanova/shrimp-prices)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import random
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import stats
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
import sklearn
from sklearn.linear_model import LinearRegression
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
path = '../input/shrimp-prices/shrimp-prices.csv'
df = pd.read_csv(path, delimiter=',')
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

In [1]:
df.tail(3)

In [1]:
df.dtypes

In [1]:
df.Month = df.Month.apply(pd.to_datetime) # pd.to_datetime(df['Date'])
df.columns = ["Date", "Price", "Change"]
df.tail(3)

In [1]:
df.dtypes

In [1]:
from datetime import datetime
# Interesting milestones
unersupply1 = {'date':datetime(year=2014, month=8, day=1), 'label':'Peak 1', 'color':'green'}
oversupply1 = {'date':datetime(year=2015, month=9, day=1), 'label':'Valley 1', 'color':'red'}
milestones  = [unersupply1, oversupply1]
# Create the datetime line plot
plt.figure() # figsize=(WIDTH, HEIGHT)
legends={"Price":"Price"}
sns.lineplot(x=df['Date'].rename(legends), y=df['Price'], label=legends['Price'])
# Plot the different milestones
if milestones:
    minimum = df['Price'].min()
    maximum = df['Price'].max()
    for milestone in milestones:
        plt.plot([milestone['date'], milestone['date']], [minimum, maximum], color=milestone['color'], linestyle='dashed', ms=10)
        plt.text(milestone['date'], minimum, milestone['label'], color=milestone['color'], rotation=45)
plt.ylabel('Price')
title = 'Shrimp Prices (US Dollars per Kilogram) 2011-2021'
plt.title(title)
plt.show()

In [1]:
sns.jointplot(data=df, x="Date", y="Price")

In [1]:
df['day'] = df['Date'].apply(lambda x:x.day)
df['month'] = df['Date'].apply(lambda x:x.month)
df['year'] = df['Date'].apply(lambda x:x.year)
df['dayofweek'] = df['Date'].apply(lambda x:x.dayofweek)
#df['weekofyear'] = df['Date'].apply(lambda x:x.isocalendar().week)
#df['is_weekend'] = df['Date'].apply(lambda x:x.dayofweek // 5)
df.drop('Date',axis=1,inplace=True)
df.dtypes

In [1]:
df.head(3)

In [1]:
ax = sns.distplot(df['Price']) # histogram distribution

In [1]:
plt.figure() # figsize=(9,5)
sns.countplot(df['month'])
plt.title('Monthwise Distribution of Sales',fontdict={'fontsize':25});

## Splitting data into training and test set

In [1]:
from sklearn.model_selection import train_test_split
target = df['Price']
X_train, X_test, y_train, y_test = train_test_split(df,target,test_size=0.30)
print(f"X_train.shape: {X_train.shape}, X_test.shape: {X_test.shape}, y_train.shape: {y_train.shape}, y_test.shape: {y_test.shape}")

## Select and evaluate the model

In [1]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
model = XGBRegressor()
model.fit(X_train, y_train)
Y_pred = model.predict(X_test)
score = model.score(X_train, y_train)
print('Training Score:', score)
score = model.score(X_test, y_test)
print('Testing Score:', score)
output = pd.DataFrame({'Predicted':Y_pred})

In [1]:
mae = np.round(mean_absolute_error(y_test,Y_pred),3)
print('Mean Absolute Error:', mae)
mse = np.round(mean_squared_error(y_test,Y_pred),3)
print('Mean Squared Error:', mse)
score = np.round(r2_score(y_test,Y_pred),3)
print('R2 Score:', score)

In [1]:
output.head(3)

## ARIMA (AutoRegressive Integrated Moving Average)

In [1]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA # Autoregressive integrated moving average

In [1]:
df = pd.read_csv(path, delimiter=',')
df.drop('Change',axis=1,inplace=True)
df.Month = df.Month.apply(pd.to_datetime) # pd.to_datetime(df['Date'])
df.set_index('Month')
index='Month'
target='Price'
indexedDataset = df.set_index([index]) # date month

From the plot below, we can see that there is a Trend component in the series. 

Hence, we now check for stationarity of the data

In [1]:
## plot graph
plt.xlabel(index)
plt.ylabel(target)
plt.plot(indexedDataset)

### Determine rolling statistics

In [1]:
rolmean = indexedDataset.rolling(window=12).mean() # window size 12 denotes 12 months, giving rolling mean at yearly level
rolmean

In [1]:
rolstd = indexedDataset.rolling(window=12).std()
rolstd

In [1]:
# Plot rolling statistics
orig = plt.plot(indexedDataset, color='blue', label='Original')
mean = plt.plot(rolmean, color='red', label='Rolling Mean')
std = plt.plot(rolstd, color='black', label='Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

From the above graph, we see that rolling mean itself has a trend component even though rolling standard deviation is fairly constant with time. 

For our time series to be stationary, we need to ensure that both the rolling statistics ie: mean & std. dev. remain time invariant or constant with time. 

Thus the curves for both of them have to be parallel to the x-axis, which in our case is not so.

To further augment our hypothesis that the time series is not stationary, let us perform the ADCF test.

In [1]:
# Perform Augmented Dickey–Fuller test:
print('Results of Dickey Fuller Test:')
results = adfuller(indexedDataset, autolag='AIC')
dfoutput = pd.Series(results[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in results[4].items():
    dfoutput['Critical Value (%s)'%key] = value
dfoutput

For a Time series to be stationary, its ADCF test should have:

p-value to be low (according to the null hypothesis)
The critical values at 1%,5%,10% confidence intervals should be as close as possible to the Test Statistics

From the above ADCF test result, we see that p-value(at max can be 1.0) is very large. 

Also critical values are no where close to the Test Statistics. 

Hence, we can safely say that our Time Series at the moment is not stationary

Data Transformation to achieve Stationarity 

There are a couple of ways to achieve stationarity through data transformation like taking log 10
 , loge
 , square, square root, cube, cube root, exponential decay, time shift and so on ...

In our notebook, lets start of with log transformations. 

Our objective is to remove the trend component. 

Hence, flatter curves( ie: paralle to x-axis) for time series and rolling mean after taking log would say that our data transformation did a good job.

## Log Scale Transformation

In [1]:
# Estimating trend
indexedDataset_logScale = np.log(indexedDataset)
plt.plot(indexedDataset_logScale)

In [1]:
# The below transformation is required to make series stationary
movingAverage = indexedDataset_logScale.rolling(window=12).mean()
movingSTD = indexedDataset_logScale.rolling(window=12).std()
plt.plot(indexedDataset_logScale)
plt.plot(movingAverage, color='red')

From above graph, we see that even though rolling mean is not stationary, it is still better than the previous case, where no transfromation were applied to series. So we can atleast say that we are heading in the right direction.

We know from above graph that both the Time series with log scale as well as its moving average have a trend component. 

Thus we can apply a elementary intuition: subtraction one from the other should remove the trend component of both. Its like:

logscaleL=stationarypart(L1)+trend(LT)
 
movingavgoflogscaleA=stationarypart(A1)+trend(AT)
 
resultseriesR=L−A=(L1+LT)−(A1+AT)=(L1−A1)+(LT−AT)
 

Since, L & A are series & it moving avg, their trend will be more or less same, Hence
LT-AT nearly equals to 0

Thus trend component will be almost removed. And we have,

R=L1−A1
 , our final non-trend curve

In [1]:
datasetLogScaleMinusMovingAverage = indexedDataset_logScale - movingAverage
datasetLogScaleMinusMovingAverage.head(12)

# Remove NAN values
datasetLogScaleMinusMovingAverage.dropna(inplace=True)
datasetLogScaleMinusMovingAverage.head(10)

In [1]:
def test_stationarity(timeseries):
    #Determine rolling statistics
    movingAverage = timeseries.rolling(window=12).mean()
    movingSTD = timeseries.rolling(window=12).std()
    #Plot rolling statistics
    orig = plt.plot(timeseries, color='blue', label='Original')
    mean = plt.plot(movingAverage, color='red', label='Rolling Mean')
    std = plt.plot(movingSTD, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    # Perform Dickey–Fuller test:
    print('Results of Dickey Fuller Test:')
    dftest = adfuller(timeseries[target], autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
test_stationarity(datasetLogScaleMinusMovingAverage)

From above graph, we observe that our intuition that "subtracting two related series having similar trend components will make the result stationary" is true. 

We find that:

p-value has reduced from 0.99 to 0.022.

The critical values at 1%,5%,10% confidence intervals are pretty close to the Test Statistic. 

Thus, from above 2 points, we can say that our given series is stationary.

But, in the spirit of getting higher accuracy, let us explore & try to find a better scale than our current log.

Let us try out Exponential decay.

For further info, refer to my answer 12 at the top of the notebook on it.

## Exponential Decay Transformation

In [1]:
exponentialDecayWeightedAverage = indexedDataset_logScale.ewm(halflife=12, min_periods=0, adjust=True).mean()
plt.plot(indexedDataset_logScale)
plt.plot(exponentialDecayWeightedAverage, color='red')

From above graph, it seems that exponential decay is not holding any advantage over log scale as both the corresponding curves are similar. 

But, in statistics, inferences cannot be drawn simply by looking at the curves. 

Hence, we perform the ADCF test again on the decay series below.

In [1]:
datasetLogScaleMinusExponentialMovingAverage = indexedDataset_logScale - exponentialDecayWeightedAverage
test_stationarity(datasetLogScaleMinusExponentialMovingAverage)

We observe that the Time Series is stationary & also the series for moving avg & std. dev. is almost parallel to x-axis thus they also have no trend.

Also,

p-value has decreased from 0.022 to 0.005.

Test Statistic value is very much closer to the Critical values.

Both the points say that our current transformation is better than the previous logarithmic transformation. 

Even though, we couldn't observe any differences by visually looking at the graphs, the tests confirmed decay to be much better.

But lets try one more time & find if an even better solution exists. 

We will try out the simple time shift technique, which is simply:

Given a set of observation on the time series:
x0,x1,x2,x3,....xn
 

The shifted values will be:
null,x0,x1,x2,....xn
  <---- basically all xi's shifted by 1 pos to right

Thus, the time series with time shifted values are:
null,(x1−x0),(x2−x1),(x3−x2),(x4−x3),....(xn−xn−1)

## Time Shift Transformation 

In [1]:
# Time Shift Transformation
datasetLogDiffShifting = indexedDataset_logScale - indexedDataset_logScale.shift()
plt.plot(datasetLogDiffShifting)

In [1]:
datasetLogDiffShifting.dropna(inplace=True)
test_stationarity(datasetLogDiffShifting)

From above 2 graphs, we can see that, visually this is the best result as our series along with rolling statistic values of moving avg & moving std. dev. is very much flat & stationary. 

But, the ADCF test shows us that:

p-value of 0.07 is not as good as 0.005 of exponential decay.

Test Statistic value not as close to the critical values as that for exponential decay.

We have thus tried out 3 different transformation: log, exp decay & time shift. 

For simplicity, we will go with the log scale. 

The reason for doing this is that we can revert back to the original scale during forecasting.

Let us now break down the 3 components of the log scale series using a system libary function. 

Once, we separate our the components, we can simply ignore trend & seasonality and check on the nature of the residual part.

In [1]:
decomposition = seasonal_decompose(indexedDataset_logScale) 

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(indexedDataset_logScale, label='Original')
plt.legend(loc='best')

plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')

plt.subplot(411)
plt.plot(seasonal, label='Seasonality')
plt.legend(loc='best')

plt.subplot(411)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')

plt.tight_layout()

In [1]:
# there can be cases where an observation simply consisted of trend & seasonality. 
# In that case, there won't be 
# any residual component & that would be a null or NaN.
# Hence, we also remove such cases.
decomposedLogData = residual
decomposedLogData.dropna(inplace=True)
# test_stationarity(decomposedLogData)

Plotting ACF & PACF

In [1]:
#ACF & PACF plots

lag_acf = acf(datasetLogDiffShifting, nlags=20)
lag_pacf = pacf(datasetLogDiffShifting, nlags=20, method='ols')

#Plot ACF:
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.title('Autocorrelation Function')            

#Plot PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function')
            
plt.tight_layout()

From the ACF graph, we see that curve touches y=0.0 line at x=2. 

Thus, from theory, Q = 2 From the PACF graph, we see that curve touches y=0.0 line at x=2. 

Thus, from theory, P = 2

ARIMA is AR + I + MA. 

Before, we see an ARIMA model, let us check the results of the individual AR & MA model. Note that, these models will give a value of RSS.

Lower RSS values indicate a better model.

Thanks [freespirit08](https://www.kaggle.com/freespirit08/time-series-for-beginners-with-arima)